### test

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 시각화 설정
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# train 데이터 로드
df_train = pd.read_csv('dataview/data/train/train.csv', encoding="utf-8")

# '시점' 처리 함수 (train 데이터)
def process_date_train(date_str):
    if pd.isna(date_str):
        return pd.NaT
    if isinstance(date_str, str):
        date_str = date_str.replace('상순', '05').replace('중순', '15').replace('하순', '25')
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
    return pd.to_datetime(date_str, errors='coerce')

# '시점' 처리 함수 (test 데이터)
def process_date_test(test_date, base_date):
    if test_date == 'T':
        return base_date
    if 'T-' in test_date:
        step = int(test_date.split('-')[1].replace('순', ''))
        return base_date - pd.DateOffset(days=step*10)
    return pd.NaT

# train 데이터 시점 처리
df_train['시점'] = df_train['시점'].apply(process_date_train)

# '평균가격(원)'과 '평년 평균가격(원)'에서 0을 NaN으로 처리 (결측값으로 간주)
df_train[['평균가격(원)', '평년 평균가격(원)']] = df_train[['평균가격(원)', '평년 평균가격(원)']].replace(0, np.nan)

# 결측치 처리 함수들
def fill_missing_chili_test(row, df, ratio_yanggeon, ratio_hwagun):
    if pd.isnull(row['평균가격(원)']):
        if row['품종명'] == '햇살양건':
            yanggeon_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '양건')]['평균가격(원)'].values
            if len(yanggeon_value) > 0 and not pd.isnull(yanggeon_value[0]):
                return yanggeon_value[0] * ratio_yanggeon
        elif row['품종명'] == '햇살화건':
            hwagun_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '화건')]['평균가격(원)'].values
            if len(hwagun_value) > 0 and not pd.isnull(hwagun_value[0]):
                return hwagun_value[0] * ratio_hwagun
        elif row['품종명'] == '양건':
            sunyanggeon_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '햇살양건')]['평균가격(원)'].values
            if len(sunyanggeon_value) > 0 and not pd.isnull(sunyanggeon_value[0]):
                return sunyanggeon_value[0] / ratio_yanggeon
        elif row['품종명'] == '화건':
            sunhwagun_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '햇살화건')]['평균가격(원)'].values
            if len(sunhwagun_value) > 0 and not pd.isnull(sunhwagun_value[0]):
                return sunhwagun_value[0] / ratio_hwagun
    return row['평균가격(원)']

def fill_missing_average_price(row, df, ratio_yanggeon, ratio_hwagun):
    if pd.isnull(row['평년 평균가격(원)']):
        if row['품종명'] == '햇살양건':
            yanggeon_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '양건')]['평년 평균가격(원)'].values
            if len(yanggeon_value) > 0 and not pd.isnull(yanggeon_value[0]):
                return yanggeon_value[0] * ratio_yanggeon
        elif row['품종명'] == '햇살화건':
            hwagun_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '화건')]['평년 평균가격(원)'].values
            if len(hwagun_value) > 0 and not pd.isnull(hwagun_value[0]):
                return hwagun_value[0] * ratio_hwagun
        elif row['품종명'] == '양건':
            sunyanggeon_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '햇살양건')]['평년 평균가격(원)'].values
            if len(sunyanggeon_value) > 0 and not pd.isnull(sunyanggeon_value[0]):
                return sunyanggeon_value[0] / ratio_yanggeon
        elif row['품종명'] == '화건':
            sunhwagun_value = df[(df['시점'] == row['시점']) & (df['품종명'] == '햇살화건')]['평년 평균가격(원)'].values
            if len(sunhwagun_value) > 0 and not pd.isnull(sunhwagun_value[0]):
                return sunhwagun_value[0] / ratio_hwagun
    return row['평년 평균가격(원)']

def calculate_ratios_test(df, crop, varieties):
    ratios = {}
    base_variety = varieties[0]
    base_mean = df[df['품종명'] == base_variety]['평균가격(원)'].mean()

    for variety in varieties[1:]:
        variety_mean = df[df['품종명'] == variety]['평균가격(원)'].mean()
        if base_mean != 0 and not np.isnan(base_mean) and not np.isnan(variety_mean):
            ratios[variety] = variety_mean / base_mean

    return ratios, base_variety

def fill_missing_by_crop_test(row, crop, varieties, ratios, base_variety, df):
    if pd.isnull(row['평균가격(원)']) and row['품종명'] in varieties:
        if row['품종명'] == base_variety:
            other_varieties = [v for v in varieties if v != base_variety]
            for other in other_varieties:
                other_value = df[(df['시점'] == row['시점']) & (df['품종명'] == other)]['평균가격(원)'].values
                if len(other_value) > 0 and not pd.isnull(other_value[0]):
                    return other_value[0] / ratios.get(other, 1)
        else:
            base_value = df[(df['시점'] == row['시점']) & (df['품종명'] == base_variety)]['평균가격(원)'].values
            if len(base_value) > 0 and not pd.isnull(base_value[0]):
                return base_value[0] * ratios.get(row['품종명'], 1)

    return row['평균가격(원)']

def fill_missing_average_by_crop_test(row, crop, varieties, ratios, base_variety, df):
    if pd.isnull(row['평년 평균가격(원)']) and row['품종명'] in varieties:
        if row['품종명'] == base_variety:
            other_varieties = [v for v in varieties if v != base_variety]
            for other in other_varieties:
                other_value = df[(df['시점'] == row['시점']) & (df['품종명'] == other)]['평년 평균가격(원)'].values
                if len(other_value) > 0 and not pd.isnull(other_value[0]):
                    return other_value[0] / ratios.get(other, 1)
        else:
            base_value = df[(df['시점'] == row['시점']) & (df['품종명'] == base_variety)]['평년 평균가격(원)'].values
            if len(base_value) > 0 and not pd.isnull(base_value[0]):
                return base_value[0] * ratios.get(row['품종명'], 1)

    return row['평년 평균가격(원)']

# 품목별 품종 그룹 정의
crop_varieties = {
    '배추': ['배추', '쌈배추', '알배기배추', '얼갈이배추', '봄동배추', '절임배추'],
    '무': ['열무', '다발무', '무'],
    '양파': ['양파', '저장양파', '조생양파', '양파(햇)', '자주양파',    '양파수입'],
    '사과': ['후지', '홍로', '쓰가루'],
    '배': ['신고', '원황'],
    '건고추': ['화건', '햇산양건', '양건'],
    '깐마늘': ['깐마늘(국산)'],
    '감자': ['감자 수미', '감자 수미(햇)', '감자', '감자 조풍', '감자 대지', '감자 두백', '감자 수미(저장)', '홍감자', '감자 수입'],
    '대파': ['대파(일반)', '실파', '깐쪽파', '쪽파', '대파수입'],
    '상추': ['청', '적']
}

# 건고추 비율 계산
ratio_yanggeon_test = df_train[df_train['품종명'] == '햇산양건']['평균가격(원)'].mean() / df_train[df_train['품종명'] == '양건']['평균가격(원)'].mean()
ratio_hwagun_test = df_train[df_train['품종명'] == '햇산화건']['평균가격(원)'].mean() / df_train[df_train['품종명'] == '화건']['평균가격(원)'].mean()

# 폴더 생성 (결과 저장 경로)
output_folder = "test_preprocess"
os.makedirs(output_folder, exist_ok=True)

# 각 테스트 파일 처리
for i in range(25):
    test_file = f"dataview/data/test/TEST_{i:02d}.csv"
    output_file = f"{output_folder}/Test_{i:02d}.csv"
    
    # test 데이터 로드
    df_test = pd.read_csv(test_file, encoding="utf-8")

    # test 데이터 시점 변환 (train 데이터의 최대 시점을 기준으로)
    train_max_date = df_train['시점'].max()
    df_test['시점'] = df_test['시점'].apply(lambda x: process_date_test(x, train_max_date))

    # '평균가격(원)'과 '평년 평균가격(원)'에서 0을 NaN으로 처리 (결측값으로 간주)
    df_test[['평균가격(원)', '평년 평균가격(원)']] = df_test[['평균가격(원)', '평년 평균가격(원)']].replace(0, np.nan)

    # 결측치 처리 전 데이터를 따로 저장해둠 (처리 전 시각화를 위해)
    df_test_before = df_test.copy()

    # 결측치 처리 전 개수 확인
    missing_before_price = df_test['평균가격(원)'].isnull().sum()
    missing_before_avg_price = df_test['평년 평균가격(원)'].isnull().sum()
    print(f"{test_file} 결측치 처리 전 - '평균가격(원)': {missing_before_price}, '평년 평균가격(원)': {missing_before_avg_price}")
    
    # 각 품목에 대한 결측치 처리 적용
    for crop, varieties in crop_varieties.items():
        if crop == '건고추':
            df_test.loc[df_test['품목명'] == crop, '평균가격(원)'] = df_test[df_test['품목명'] == crop].apply(
                lambda row: fill_missing_chili_test(row, df_train, ratio_yanggeon_test, ratio_hwagun_test), axis=1
            )
            df_test.loc[df_test['품목명'] == crop, '평년 평균가격(원)'] = df_test[df_test['품목명'] == crop].apply(
                lambda row: fill_missing_average_price(row, df_train, ratio_yanggeon_test, ratio_hwagun_test), axis=1
            )
        else:
            ratios, base_variety = calculate_ratios_test(df_train, crop, varieties)
            df_test.loc[df_test['품목명'] == crop, '평균가격(원)'] = df_test[df_test['품목명'] == crop].apply(
                lambda row: fill_missing_by_crop_test(row, crop, varieties, ratios, base_variety, df_train), axis=1
            )
            df_test.loc[df_test['품목명'] == crop, '평년 평균가격(원)'] = df_test[df_test['품목명'] == crop].apply(
                lambda row: fill_missing_average_by_crop_test(row, crop, varieties, ratios, base_variety, df_train), axis=1
            )
    
    # 결측치 처리 후 개수 확인
    missing_after_price = df_test['평균가격(원)'].isnull().sum()
    missing_after_avg_price = df_test['평년 평균가격(원)'].isnull().sum()
    print(f"{test_file} 결측치 처리 후 - '평균가격(원)': {missing_after_price}, '평년 평균가격(원)': {missing_after_avg_price}")

    # 처리된 데이터를 저장
    df_test.to_csv(output_file, index=False)

    # 시각화: 결측치 처리 전/후 비교
    plt.figure(figsize=(12, 6))
    plt.scatter(df_test_before['시점'], df_test_before['평균가격(원)'], alpha=0.5, label='처리 전', color='blue', marker='o')
    plt.scatter(df_test['시점'], df_test['평균가격(원)'], alpha=0.5, label='처리 후', color='red', marker='x')
    plt.title(f'Test_{i:02d} 평균가격(원) 처리 전/후 비교')
    plt.xlabel('시점')
    plt.ylabel('평균가격(원)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 저장된 파일 및 시각화 결과 확인
    print(f"결과 저장 완료: {output_file}")

